<a href="https://colab.research.google.com/github/steve122192/TA_review_scraper/blob/main/Initial_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Udel Meeting Prep



> Are they technincal people? will notebook be okay?

> Does Jeff want to take the lead? will you want us to present something or just answer questions as they come along

# Data Sources


# Data
### EDA


> Subset data into pre and post covid sets, then negative and positive. Get word clouds for each. Does anything stand out?


> Get postive vs negative review ratio per month. How has it changed over time? Did covid seem to have an affect?

### Sentiment Analysis

> Train a binary sentiment classfier pre and post covid. Get feature importancees. Anything interesting?

### Topic Modeling
> Model entire set. Anything interesting pop out? Dig deeper and run on subsets.





In [ ]:
import pyLDAvis.gensim
import spacy
from spacy.tokenizer import Tokenizer
import en_core_web_lg
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary
from gensim import corpora

In [1]:
from google.colab import files
uploaded = files.upload()

Saving restos_wilmington_area.csv to restos_wilmington_area.csv


In [35]:
import pandas as pd
df = pd.read_csv('restos_wilmington_area.csv', index_col=0)

df

,rating,text,date,area
0,5,Great meal and an even greater job by Big Fish...,Nov 2020,"Wilmington, DE"
1,4,It was nice to see somebody operating under al...,Jul 2020,"Wilmington, DE"
2,4,Husband and I hadn't been to a restaurant sinc...,Aug 2020,"Wilmington, DE"
3,4,Overall a decent placeDrinks are good. Apple c...,Oct 2020,"Wilmington, DE"
4,1,We waited about an hour for an outside table (...,Oct 2020,"Wilmington, DE"
...,...,...,...,...
2509,4,Went here with my two kids and wife for lunch....,Aug 2014,"Wilmington, DE"
2510,3,I think the food is great and better than Oliv...,Jun 2014,"Wilmington, DE"
2511,4,Last night was my first time here and I must s...,Jul 2014,"Wilmington, DE"
2512,2,I ordered gnocchi which is advertised at 12.95...,Apr 2016,"Wilmington, DE"


In [42]:
df  = df[df['rating']<3]

In [ ]:
nlp = en_core_web_lg.load()
#nlp = spacy.load("en_core_web_lg")

def tokenize(document):
    
    doc = nlp(document)
    list(doc.noun_chunks)
    for noun_phrase in list(doc.noun_chunks):
      noun_phrase.merge(noun_phrase.root.tag_, noun_phrase.root.lemma_, noun_phrase.root.ent_type_)
    
    return [token.lemma_.strip().lower() for token in doc if (token.is_stop != True) and (token.is_punct != True) and (token.is_digit != True) and (token.is_space != True) and (token.is_currency != True) and (token.like_num != True)]

tokens = []
for doc in df['text']:
    x = tokenize(doc)
    tokens.append(x)
    
df['tokens'] = tokens
df

In [5]:
tfidf = TfidfVectorizer(stop_words='english', 
                        tokenizer=tokenize, 
                        #min_df=.0,
                        max_df=.90,
                        max_features=5000,
                        ngram_range=(1,2),
                       )
dtm = tfidf.fit_transform(df['text'])
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())
dtm

,+,-pron-,1st,2nd,4th,6:30,=,able,absolute,absolutely,absolutely amazing,absolutely delicious,absolutely love,absolutely perfect,absolutely return,absolutely wonderful,accent,accept,acceptable,accessible,accident,accommodate,accommodating,accompaniment,accompany,account,accurate,achieve,acknowledge,acme,acoustic,act,act like,actively,actual,actually,actually pretty,add,add lot,add nice,...,worth try,worth visit,worth wait,worthy,wound,wow,wrap,write,write home,write review,wrong,wrong place,y,ya,yeah,year,year ago,year eat,year eve,year food,year old,year round,yell,yellow,yellow curry,yelp,yelp review,yelper,yes,yesterday,york,yorkshire,yorkshire pudding,young,yuck,yum,yummy,zucchini,~,~ good
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.101732,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2509,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.190511,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2510,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2511,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2512,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [44]:
id2word = corpora.Dictionary(df['tokens'])

In [45]:
corpus = [id2word.doc2bow(text) for text in df['tokens']]

In [50]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=100,
                   workers=4,
                   num_topics = 20 # You can change this parameter
                  )

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [17]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 12.0MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=0244c0adcbf13b4009e9b1de175b145a75f98d93b70bea201fbb280837288ace
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [51]:


pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18     0.009176 -0.010789       1        1  12.332205
2     -0.000322  0.004754       2        1  11.423640
8     -0.002778 -0.039034       3        1   7.827016
19     0.021334 -0.015568       4        1   7.256728
9     -0.015230 -0.038869       5        1   6.027045
13     0.029147  0.017139       6        1   5.338926
12     0.005500 -0.065121       7        1   4.676838
4      0.059273 -0.049630       8        1   4.623830
10     0.023009 -0.035676       9        1   4.555956
5      0.016242 -0.000683      10        1   4.232702
7      0.003220  0.026578      11        1   4.150016
3      0.046712  0.006774      12        1   4.122967
0     -0.017110 -0.061302      13        1   3.719134
17    -0.005605  0.054092      14        1   3.647161
1      0.010494  0.082143      15        1   3.631279
14     0.028325  0.101478      16        1   3.117220
15    -0.074111 -0.002676      17        1   2.918396
6      0.044435 -0.000254      18        1   2.652402
16    -0.035887  0.019819      19        1   2.363858
11    -0.145825  0.006826      20        1   1.382680, topic_info=            Term        Freq       Total Category  logprob  loglift
31          food  352.000000  352.000000  Default  30.0000  30.0000
36          good  232.000000  232.000000  Default  29.0000  29.0000
46          like  182.000000  182.000000  Default  28.0000  28.0000
3535     british   19.000000   19.000000  Default  27.0000  27.0000
74          time  171.000000  171.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
47          look    1.314082   79.365429  Topic20  -5.5150   0.1802
172          bad    1.319408   93.226848  Topic20  -5.5109   0.0233
236         fish    1.215977   49.755707  Topic20  -5.5926   0.5696
62    restaurant    1.304586  147.227543  Topic20  -5.5222  -0.4449
86         check    1.217747   58.351627  Topic20  -5.5911   0.4117

[1731 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
3808     20  0.726728  -spaghetti
1310     12  0.636716    .looking
3378      7  0.570387        .the
2958      3  0.569312       1/5th
851      16  0.625731      1045pm
...     ...       ...         ...
1101      1  0.422556       young
1101      7  0.281704       young
1101     12  0.140852       young
1707     15  0.575171      zapper
3110      4  0.629267        ~$13

[4715 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 3, 9, 20, 10, 14, 13, 5, 11, 6, 8, 4, 1, 18, 2, 15, 16, 7, 17, 12])